In [1]:
"""Visualization utilies."""

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def show_points(points, title):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_title(title)
    ax.set_xlim3d([-2, 2])
    ax.set_ylim3d([-2, 2])
    ax.set_zlim3d([0, 4])
    ax.scatter(points[:, 0], points[:, 2], points[:, 1])

def compare_points(points1, points2):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_xlim3d([-2, 2])
    ax.set_ylim3d([-2, 2])
    ax.set_zlim3d([0, 4])
    ax.scatter(points1[:, 0], points1[:, 2], points1[:, 1])
    ax.scatter(points2[:, 0], points2[:, 2], points2[:, 1])

In [2]:
# Load the canonical-space object point clouds
import trimesh
import collada
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

def fps_downsample(points, number_of_points_to_sample):
  selected_points = np.zeros((number_of_points_to_sample, 3))
  dist = np.ones(points.shape[0]) * np.inf # distance to the selected set
  for i in tqdm(range(number_of_points_to_sample)):
      # pick the point with max dist
      idx = np.argmax(dist)
      selected_points[i] = points[idx]
      dist_ = ((points - selected_points[i]) ** 2).sum(-1)
      dist = np.minimum(dist, dist_)
  return selected_points

object_models_file = "models/objects_v1.csv"
object_models_info = pd.read_csv(object_models_file)
object_models_name = object_models_info["object"].to_list()
object_models_location = object_models_info["location"].to_list()
object_models_num = len(object_models_name)


object_models = {}
for object_name, object_mesh_dir in zip(object_models_name, object_models_location):
  object_mesh_path = os.path.join(object_mesh_dir, "visual_meshes", "visual.dae")
  # Collada-based loading
  object_mesh = trimesh.exchange.dae.load_collada(object_mesh_path)
  mesh_name = object_mesh['graph'][0]['geometry']
  object_models[object_name] = object_mesh['geometry'][mesh_name]['vertices']
  # Trimesh-based loading
  # object_mesh = trimesh.load(object_mesh_path, force='mesh')
  # object_models[object_name] = object_mesh.vertices
  # print(object_mesh)

# object_models with key=objectName, value=objectPointCloud
# print(object_models_info)
print(object_models["jenga"])
print(len(object_models["jenga"]))
# print(np.unique(object_models["jenga"]))
# print(len(object_models))







[[-0.07499998 -0.02499997  0.01499998]
 [ 0.07499998 -0.02499997  0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [-0.07499998 -0.02499997  0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [-0.07499998  0.02499997  0.01499998]
 [ 0.07499998  0.02499997 -0.01499998]
 [ 0.07499998 -0.02499997 -0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [-0.07499998  0.02499997 -0.01499998]
 [ 0.07499998  0.02499997 -0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [-0.07499998  0.02499997  0.01499998]
 [-0.07499998  0.02499997 -0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [-0.07499998 -0.02499997  0.01499998]
 [-0.07499998  0.02499997  0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [ 0.07499998 -0.02499997 -0.01499998]
 [ 0.07499998  0.02499997 -0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [ 0.07499998 -0.02499997 -0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [ 0.07499998 -0.02499997  0.01499998]
 [ 0.07499998 -0.02499997  0.01499998]
 [-0.07499998 -0.02499997

In [3]:
"""Data loading helpers"""
import numpy as np
import json
import matplotlib.pyplot as plt
import os
from PIL import Image
import pickle
import utils
import open3d as o3d
# # Helper function for the reconstruction of the target point cloud
# rgb = np.array(Image.open(rgb_files[0])) / 255   # convert 0-255 to 0-1
# depth = np.array(Image.open(depth_files[0])) / 1000   # convert from mm to m
# label = np.array(Image.open(label_files[0]))
# meta = load_pickle(meta_files[0])
# intrinsic = meta['intrinsic']
# z = depth
# v, u = np.indices(z.shape)
# uv1 = np.stack([u + 0.5, v + 0.5, np.ones_like(z)], axis=-1)
# points_viewer = uv1 @ np.linalg.inv(intrinsic).T * z[..., None]  # [H, W, 3]
# # print(points_viewer.shape)
# # print(points_viewer.shape[0]*points_viewer.shape[1])
# # print(points_viewer)

def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def dump_json(sample, path):
  with open(path, 'w') as fp:
    json.dump(sample, fp)
  return 0

def load_json(path):
  f = open(path)
  data = json.load(f)
  return data

def get_point_cloud(depth, intrinsic):
  z = depth
  v, u = np.indices(z.shape)
  uv1 = np.stack([u + 0.5, v + 0.5, np.ones_like(z)], axis=-1)
  points_viewer = uv1 @ np.linalg.inv(intrinsic).T * z[..., None]  # [H, W, 3]
  return points_viewer

def pts_to_o3d_pcd(pts):
  """Transform to o3d pcd"""
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(pts)
  return pcd

def o3dvis(pts):
    o3d.visualization.draw_geometries([pts_to_o3d_pcd(pt) for pt in pts])
    return 0

def get_object_point_cloud(test_image_label, object_id, test_depth, intrinsic):
  # print(np.where(test_image_label==object_id))
  # print(test_image_label[327][654])
  test_image_label[np.where(test_image_label==object_id)] = 255
  # print(test_image_label[327][654])
  # print(np.where(test_image_label==255))
  test_image_label[np.where(test_image_label!=255)] = 0
  test_image_label[np.where(test_image_label==255)] = 1
  test_object_depth = test_depth * test_image_label
  test_pcd_target = get_point_cloud(test_object_depth, intrinsic)
  # (H, W, dim) = test_pcd_target.shape
  # filter out target object point cloud
  # print(test_pcd_target.shape)
  # print(test_pcd_target)
  # print((test_pcd_target[:,0]!=0)|(test_pcd_target[:,1]!=0)|(test_pcd_target[:,2]!=0))
  test_pcd_target = test_pcd_target.reshape(-1, test_pcd_target.shape[-1]) # reshape to (H*W, 3)
  test_pcd_target = test_pcd_target[(test_pcd_target[:,0]!=0)|(test_pcd_target[:,1]!=0)|(test_pcd_target[:,2]!=0)]
  return test_pcd_target


def get_meta(meta_path):
  return load_pickle(meta_path)

def get_depth(depth_path):
  return (np.array(Image.open(depth_path))/1000)

def get_label(label_path):
  return np.array(Image.open(label_path))


def dump_json(sample, path):
  with open(path, 'w') as fp:
    json.dump(sample, fp)
  return 0

def load_json(path):
  f = open(path)
  data = json.load(f)
  return data

def get_point_cloud(depth, intrinsic):
  z = depth
  v, u = np.indices(z.shape)
  uv1 = np.stack([u + 0.5, v + 0.5, np.ones_like(z)], axis=-1)
  points_viewer = uv1 @ np.linalg.inv(intrinsic).T * z[..., None]  # [H, W, 3]
  return points_viewer


def get_meta(meta_path):
  return load_pickle(meta_path)

def get_depth(depth_path):
  return (np.array(Image.open(depth_path))/1000)

def get_label(label_path):
  return np.array(Image.open(label_path))


"""Metric and visualization."""

def compute_rre(R_est: np.ndarray, R_gt: np.ndarray):
    """Compute the relative rotation error (geodesic distance of rotation)."""
    assert R_est.shape == (3, 3), 'R_est: expected shape (3, 3), received shape {}.'.format(R_est.shape)
    assert R_gt.shape == (3, 3), 'R_gt: expected shape (3, 3), received shape {}.'.format(R_gt.shape)
    # relative rotation error (RRE)
    # Rotational degree loss (not objective of optimization)
    rre = np.arccos(np.clip(0.5 * (np.trace(R_est.T @ R_gt) - 1), -1.0, 1.0))
    return rre


def compute_rte(t_est: np.ndarray, t_gt: np.ndarray):
    assert t_est.shape == (3,), 't_est: expected shape (3,), received shape {}.'.format(t_est.shape)
    assert t_gt.shape == (3,), 't_gt: expected shape (3,), received shape {}.'.format(t_gt.shape)
    # relative translation error (RTE)
    rte = np.linalg.norm(t_est - t_gt) # Resembling MSE loss
    return rte




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [8]:
"""Visualization on comparing between train pose-transformed point clouds."""
import open3d as o3d
import numpy as np
import time
import copy

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

def get_o3d_icp_tensor(source_pcd, target_pcd):
  # T = icp(source_pcd, target_pcd, 150)
  source_pcd_o3d, target_pcd_o3d = o3d.t.geometry.PointCloud(), o3d.t.geometry.PointCloud()
  # print(source_pcd)
  source_pcd_o3d.point.positions = o3d.core.Tensor(source_pcd, dtype=o3d.core.Dtype.Float64)
  # source_pcd_o3d.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(knn=50))
  target_pcd_o3d.point.positions = o3d.core.Tensor(target_pcd, dtype=o3d.core.Dtype.Float64)
  # Search distance for Nearest Neighbour Search [Hybrid-Search is used].
  max_correspondence_distance = 1
  init = o3d.core.Tensor.eye(4, o3d.core.Dtype.Float64)
  # Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
  treg = o3d.cpu.pybind.t.pipelines.registration
  estimation = treg.TransformationEstimationPointToPoint()
  callback_after_iteration = lambda updated_result_dict : print("Iteration Index: {}, Fitness: {}, Inlier RMSE: {},".format(
    updated_result_dict["iteration_index"].item(), \
    updated_result_dict["fitness"].item(), \
    updated_result_dict["inlier_rmse"].item())) \

  # Convergence-Criteria for Vanilla ICP
  criteria = treg.ICPConvergenceCriteria(relative_fitness=0.0000000000001,
                      relative_rmse=0.0000000000001,
                      max_iteration=100)

  # Down-sampling voxel-size.
  # voxel_size = 0.0016625
  # voxel_size = 0.003125
  voxel_size = 0.00625
  # voxel_size = 0.0125

  # Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
  save_loss_log = True
  s = time.time()

  # voxel_size = 0.05  # means 5cm for this dataset
  # source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, source_pcd_o3d, target_pcd_o3d)
  # result_ransac = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
  # init = o3d.core.Tensor(result_ransac.transformation, dtype=o3d.core.Dtype.Float64)

  registration_icp = treg.icp(source_pcd_o3d, target_pcd_o3d, max_correspondence_distance, init, estimation, criteria, voxel_size, callback_after_iteration)
  icp_time = time.time() - s
  T = registration_icp.transformation
  T = T.numpy()
  return T

def execute_fast_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
  distance_threshold = voxel_size * 0.5
  print(":: Apply fast global registration with distance threshold %.3f" \
          % distance_threshold)
  result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
      source_down, target_down, source_fpfh, target_fpfh,
      o3d.pipelines.registration.FastGlobalRegistrationOption(
          maximum_correspondence_distance=distance_threshold))
  return result

def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size, init):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result




















icp_global_voxel_size = 0.05
icp_dist_threshold = 5
init_random_time = 300
max_preproc_nn_normal = 50
max_preproc_nn_fpfh = 100
tgSrcFactor = 1.3
dist_voxel_factor = 3
fps_dnsamp_factor = 2000
cdistFrac = 0.1
heavy_dnsamp_frac = 9/14
# heavy_dnsamp_frac = 5/7
# heavy_dnsamp_frac = 4/7


def get_o3d_icp(source_pcd, target_pcd, init=np.eye(4), icp_dist_threshold=icp_dist_threshold):
  # T = icp(source_pcd, target_pcd, 150)
  source_pcd_o3d, target_pcd_o3d = o3d.geometry.PointCloud(), o3d.geometry.PointCloud()
  # print(source_pcd)
  source_pcd_o3d.points = o3d.utility.Vector3dVector(source_pcd)
  # source_pcd_o3d.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(knn=50))
  target_pcd_o3d.points = o3d.utility.Vector3dVector(target_pcd)
  # target_pcd_o3d.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(knn=50))
  # print(source_pcd_o3d)
  # print(target_pcd_o3d)
  # init = np.random.rand(4,4)
  # init = -1 + init * 2
  # init[-1] = np.zeros(4)
  # print(init)
  # init[0,0] = 2
  T = o3d.pipelines.registration.registration_icp( \
        source_pcd_o3d, target_pcd_o3d, icp_dist_threshold, init, \
        o3d.pipelines.registration.TransformationEstimationPointToPoint(), \
        # o3d.pipelines.registration.TransformationEstimationPointToPlane(), \
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=0.00001, \
                                 relative_rmse=0.00001, \
                                 max_iteration=500))
  # T = icp(source_pcd, target_pcd)
  # print(T.inlier_rmse)
  # print(T.fitness)
  # print(T.correspondence_set)
  return T

# tgSrcFactorDict = {
#    58: 1.2

# }

def half_voxel_dnsample(source, voxel_size, heavy_dnsamp_frac=heavy_dnsamp_frac):
  pts = source.points
  size = len(pts)
  # print(pts.shape)
  first_half = o3d_Vec_to_o3d_pcd(pts[:int(size*heavy_dnsamp_frac)])
  first_half = first_half.voxel_down_sample(voxel_size=voxel_size) # heavily down-sampled side
  secn_half = o3d_Vec_to_o3d_pcd(pts[int(size*heavy_dnsamp_frac):])
  secn_half = secn_half.voxel_down_sample(voxel_size=voxel_size*0.1) # weakly down-sampled side

  first_half_pts = np.asarray(first_half.points) 
  secn_half_pts = np.asarray(secn_half.points)
  full_pcd_pts = np.vstack((first_half_pts, secn_half_pts))
  full_pcd_pts = pts_to_o3d_pcd(full_pcd_pts)
  return full_pcd_pts

def preprocess_point_cloud(source, target, 
                           voxel_size, max_nn_normal=max_preproc_nn_normal, 
                           max_nn_fpfh=max_preproc_nn_fpfh, tgSrcFactor=tgSrcFactor, fps_dnsamp_factor=fps_dnsamp_factor):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    # pts = pcd.points
    # size = len(source.points)
    # if (len(target.points) > 1000):
    #   target = target.farthest_point_down_sample(len(target.points))
    size_factor = len(source.points) / len(target.points)
    if (size_factor >= 1):
      # print(len(target.points), fps_dnsamp_factor, len(target.points)//fps_dnsamp_factor)
      # src_pcd_down = source.farthest_point_down_sample(len(source.points)//fps_dnsamp_factor)
      # src_pcd_down = source.voxel_down_sample(voxel_size=0.019)
      # src_pcd_down = source.voxel_down_sample(voxel_size=voxel_size)
      src_pcd_down = half_voxel_dnsample(source, voxel_size=voxel_size)
    else: 
      src_pcd_down = source
    # src_pcd_down = source
    # tg_pcd_down = target
    tg_pcd_down = target.farthest_point_down_sample(int(tgSrcFactor*len(src_pcd_down.points)))
    # src_pcd_down = source.farthest_point_down_sample(int(size/5))
    # src_pcd_down = source


    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    src_pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=max_nn_normal))
    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    src_pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        src_pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn_fpfh))


    print(":: Estimate normal with search radius %.3f." % radius_normal)
    tg_pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=max_nn_normal))
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    tg_pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        tg_pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn_fpfh))

    return src_pcd_down, src_pcd_fpfh, tg_pcd_down, tg_pcd_fpfh

def prepare_dataset(voxel_size, source, target):
    print(":: Load two point clouds and disturb initial pose.")

    # trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
    #                          [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    # source.transform(trans_init)

    # source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    # target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    source_down, source_fpfh, target_down, target_fpfh = preprocess_point_cloud(source, target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size, dist_voxel_factor=dist_voxel_factor):
  distance_threshold = voxel_size * dist_voxel_factor
  # print(":: RANSAC registration on downsampled point clouds.")
  # print("   Since the downsampling voxel size is %.3f," % voxel_size)
  # print("   we use a liberal distance threshold %.3f." % distance_threshold)
  result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
      source_down, target_down, source_fpfh, target_fpfh, True,
      distance_threshold,
      o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
      3, [
          o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
              0.9),
          o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
              distance_threshold)
      ], o3d.pipelines.registration.RANSACConvergenceCriteria(10000000, 0.99999))
  return result


def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range

def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

def visualize_rre(Trans, rmse, cdists, optim_index):
  rmseList = np.array(rmse)
  cdistsList = np.array(cdists)
  rreList = [np.rad2deg(compute_rre((inv_extrinsic @ t)[:3,:3], gt_T_world[:3,:3])) for t in Trans]
  rteList = [compute_rte((inv_extrinsic @ t)[:3,3], gt_T_world[:3,3]) for t in Trans]
  print("The following gives rre and rte max indices and values:")
  print(np.max(rreList), np.argmax(rreList))
  print(np.min(rteList), np.argmin(rteList))
  # print(rteList[np.argmax(rreList)], np.argmax(rreList))
  rreList, rteList = np.array(rreList), np.array(rteList) 
  # print((rreList>=175))
  # print((rteList<=0.01))
  # np.intersect1d(np.where(rreList>=175), np.where(rteList<=0.01)) 
  goodList = rreList[np.intersect1d(np.where(rreList>=175), np.where(rteList<=0.01))]
  print("Good list under current parameter sets:")
  print(goodList, len(goodList))
  print("Good list rmse and cdist:")
  print(rmseList[np.intersect1d(np.where(rreList>=175), np.where(rteList<=0.01))])
  print(cdistsList[np.intersect1d(np.where(rreList>=175), np.where(rteList<=0.01))])
  print("Fractioned optimal rmse and cdist:")
  print(rmseList[optim_index])
  print(cdistsList[optim_index])
  print("Global optimal rmse and cdist, with indices:")
  print(np.min(rmseList), np.argmin(rmseList))
  print(np.min(cdistsList), np.argmin(cdistsList))
  return 0

def get_o3d_icp_with_global_registration(source_pcd, target_pcd, 
                                         voxel_size=icp_global_voxel_size, 
                                         init_random_time=init_random_time, 
                                         cdistFrac = cdistFrac):
  source_pcd_o3d, target_pcd_o3d = o3d.geometry.PointCloud(), o3d.geometry.PointCloud()
  source_pcd_o3d.points = o3d.utility.Vector3dVector(source_pcd)
  target_pcd_o3d.points = o3d.utility.Vector3dVector(target_pcd)
  source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, source_pcd_o3d, target_pcd_o3d)
  print(source, target)
  print(source_down, target_down)

  # perform multiple RANSAC inits, and choose the best amongest them
  Trans, rmse, cdists = [], [], []
  for i in tqdm(range(init_random_time)):
    result_ransac = execute_global_registration(source_down, target_down, \
                      source_fpfh, target_fpfh, \
                      voxel_size)
    # result_ransac = execute_fast_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
    # print("result_ransac: ")
    # print(result_ransac)
    # print(result_ransac.transformation.shape)
    # o3dvis([getTransPcd(source_down.points, result_ransac.transformation), target_down.points])
    # result_icp = refine_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size, result_ransac.transformation)
    # TODO: Fetch the best initialization as the result
    result_icp = get_o3d_icp(source_down.points, target_down.points, init=result_ransac.transformation)
    # print("result_icp:")
    # print(result_icp)
    tr = result_icp.transformation
    trans_source_down = copy.deepcopy(source_down)
    trans_source_down = trans_source_down.transform(tr)
    dists = target_down.compute_point_cloud_distance(trans_source_down)
    dists = np.asarray(dists)
    # print("chamfer dist:")
    # print(dists)
    # print(dists)
    Trans.append(tr)
    rmse.append(result_icp.inlier_rmse)
    cdists.append(np.sum(dists))
    # draw_registration_result(source_down, target_down, result_icp.transformation)
  print("cdist list:")
  print(cdists)
  print(np.min(cdists))
  print(np.argmin(cdists))
  o3dvis([getTransPcd(source_down.points, Trans[np.argmin(np.array(cdists)+np.array(rmse))]), target_down.points])
  # return Trans[np.argmin(rmse)]
  cdistsNorm = standardization(np.array(cdists))
  rmseNorm = standardization(np.array(rmse))
  optim_index = np.argmin((1-cdistFrac)*rmseNorm+cdistFrac*cdistsNorm)
  # visualize_rre(Trans, rmse, cdists, optim_index)
  visualize_rre(Trans, rmseNorm, cdistsNorm, optim_index)
  return Trans[np.argmin((1-cdistFrac)*rmseNorm+cdistFrac*cdistsNorm)]

def pts_to_o3d_pcd(pts):
  """Transform to o3d pcd"""
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(pts)
  return pcd

def o3d_Vec_to_o3d_pcd(pts):
  """Transform to o3d pcd"""
  pcd = o3d.geometry.PointCloud()
  pcd.points = pts
  return pcd

def o3dvis(pts):
  o3d.visualization.draw_geometries([pts_to_o3d_pcd(pt) for pt in pts])
  return 0

def getTransPcd(source_pcd, T):
  return source_pcd @ T[:3, :3].T + T[:3, 3]


# Define the data point to visualize
train_vis_varianct, train_vis_index, object_ids, vis_id = "1-1-4", 0, [35, 39, 48, 51, 58], 35
rgb_file = "datas/1-1-4_color_kinect.png"
depth_file = "datas/1-1-4_depth_kinect.png"
label_file = "datas/1-1-4_label_kinect.png"
meta_file = "datas/1-1-4_meta.pkl"


# Load scene meta info
meta_vis = get_meta(meta_file)
scales = meta_vis["scales"]
print(meta_vis.keys())
intrinsic = meta_vis["intrinsic"]
extrinsic = meta_vis["extrinsic"]
inv_extrinsic = np.linalg.inv(extrinsic)
gt_T_world = meta_vis["poses_world"][vis_id]
gt_T = extrinsic @ gt_T_world
# print(gt_T.shape)
gt_T = meta_vis["poses_world"][vis_id]
# Fetch source point cloud from model dictionary
# print(object_models)


# Reconstruct source and target point clouds (in camera frame)
print(object_name)
object_name = object_models_name[vis_id]
vis_pcd_source = object_models[object_name]
vis_pcd_source = vis_pcd_source * scales[vis_id]
vis_pcd_target = get_object_point_cloud(get_label(label_file), \
                    vis_id, \
                    get_depth(depth_file), \
                    intrinsic)

# o3dvis([vis_pcd_source])

print("src and tg len:")
print(len(vis_pcd_target), len(vis_pcd_source))


# Perform ICP
# source_pcd, target_pcd = vis_pcd_target, vis_pcd_source
source_pcd, target_pcd = vis_pcd_source, vis_pcd_target
# source_pcd = np.hstack((source_pcd, np.ones((len(source_pcd), 1))))
# target_pcd = np.hstack((target_pcd, np.ones((len(target_pcd), 1))))
# source_pcd, target_pcd = (source_pcd @ np.linalg.inv(extrinsic).T), (target_pcd @ np.linalg.inv(extrinsic).T)
# source_pcd = np.array([x[:-1]/x[-1] for x in source_pcd])
# target_pcd = np.array([x[:-1]/x[-1] for x in target_pcd])
# print(source_pcd.shape, target_pcd.shape)
# o3dvis([source_pcd, target_pcd])
T = get_o3d_icp_with_global_registration(source_pcd, target_pcd)
# T = get_o3d_icp_tensor(source_pcd, target_pcd)
# T = get_o3d_icp(source_pcd, target_pcd) # should use T.transformation to get transformation matrix!
# T = icp(source_pcd, target_pcd, 100)
T_world = inv_extrinsic @ T
TrEstiPcd = source_pcd @ T[:3, :3].T + T[:3, 3]
TrGtPcd = source_pcd @ gt_T[:3, :3].T + gt_T[:3, 3]

o3dvis([TrEstiPcd, target_pcd])


print(len(TrEstiPcd), len(TrGtPcd))
# print(TrEstiPcd)
# print(TrGtPcd)
# o3d.visualization.draw_geometries([pts_to_o3d_pcd(TrEstiPcd), pts_to_o3d_pcd(TrGtPcd)])

# T = np.eye(4)
print(source_pcd.shape)
print(target_pcd.shape)
print("GT T:")
print(gt_T)
print("GT T world:")
print(gt_T_world)
print("pred T:")
print(T)
print("pred T world:")
print(T_world)
# rre = np.rad2deg(compute_rre(T[:3, :3], gt_T[:3, :3]))
# rte = compute_rte(T[:3, 3], gt_T[:3, 3])
rre = np.rad2deg(compute_rre(T_world[:3, :3], gt_T_world[:3, :3]))
rte = compute_rte(T_world[:3, 3], gt_T_world[:3, 3])
print(f"rre={rre}, rte={rte}")
# show_points(target_pcd, "Target pcd")
# show_points(source_pcd, "Source pcd")
# show_points(TrEstiPcd, "Transformed source pcd")
# show_points(TrGtPcd, "GT Transformed source pcd")






dict_keys(['poses_world', 'extents', 'scales', 'object_ids', 'object_names', 'extrinsic', 'intrinsic'])
master_chef_can
src and tg len:
4417 36
:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
PointCloud with 36 points. PointCloud with 4417 points.
PointCloud with 36 points. PointCloud with 46 points.


100%|██████████| 300/300 [00:01<00:00, 185.45it/s]


cdist list:
[1.892937505737352, 1.8929375057373514, 1.8464653502871498, 1.8929375057373459, 1.876322193424986, 1.8929375057373463, 1.7850174779920416, 1.8464653502871424, 1.876322193424991, 1.8464653502871444, 1.7850174779920396, 1.8464653502871469, 1.8464653502871449, 1.8929375057373496, 1.8929375057373503, 1.8464653502871466, 1.8763221934249876, 1.8929375057373499, 1.8464653502871475, 1.8464653502871429, 1.846465350287149, 1.8929375057373479, 1.7996464711912648, 1.7850174779920422, 1.846465350287146, 1.846465350287145, 1.8464653502871484, 1.8929375057373494, 1.846465350287144, 1.8763221934249859, 1.8763221934249867, 1.8929375057373459, 1.892937505737348, 1.8464653502871444, 1.846465350287145, 1.846465350287143, 1.7996464711912676, 1.7996464711912676, 1.892937505737349, 1.8763221934249876, 1.8464653502871433, 1.8464653502871458, 1.8763221934249883, 1.846465350287149, 1.8763221934249859, 1.7996464711912672, 1.7850174779920427, 1.7850174779920447, 1.7850174779920431, 1.8464653502871478,

In [5]:
"""Visualize given pcd"""
train_vis_varianct, train_vis_index, object_ids, vis_id = "1-1-1", 0, [35, 39, 48, 51, 58], 58
object_name = object_models_name[vis_id]
meta_vis = get_meta(meta_file)
scales = meta_vis["scales"]
vis_pcd_source = object_models[object_name]
vis_pcd_source = vis_pcd_source * scales[vis_id]
o3d.visualization.draw_geometries([pts_to_o3d_pcd(vis_pcd_source)])


